# Polars Experiments



In [2]:

import  numpy as np
import polars as pl
from polars import col

import yfinance as yf

from mintalib.core import calc_sma, calc_atr, calc_macd
from mintalib.indicators import SMA, ATR, MACD


In [3]:

prices = yf.Ticker('AAPL').history('5y')
prices = prices.rename(columns=str.lower).rename_axis(index=str.lower)
prices = pl.from_pandas(prices, include_index=True)
prices


# Links
# https://docs.pola.rs/user-guide/expressions/user-defined-python-functions/#combining-multiple-column-values

date,open,high,low,close,volume,dividends,stock splits
"datetime[ns, America/New_York]",f64,f64,f64,f64,i64,f64,f64
2020-02-03 00:00:00 EST,73.753387,75.980774,73.24926,74.810127,173788400,0.0,0.0
2020-02-04 00:00:00 EST,76.421898,77.471367,76.014717,77.279892,136616400,0.0,0.0
2020-02-05 00:00:00 EST,78.411736,78.71228,77.304108,77.910034,118826800,0.0,0.0
2020-02-06 00:00:00 EST,78.181495,78.823776,77.62162,78.82135,105425600,0.0,0.0
2020-02-07 00:00:00 EST,78.318439,78.568673,77.256767,77.749947,117684000,0.1925,0.0
…,…,…,…,…,…,…,…
2025-01-27 00:00:00 EST,224.020004,232.149994,223.979996,229.860001,94863400,0.0,0.0
2025-01-28 00:00:00 EST,230.850006,240.190002,230.809998,238.259995,75707600,0.0,0.0
2025-01-29 00:00:00 EST,234.119995,239.860001,234.009995,239.360001,45486100,0.0,0.0


In [4]:
pstruct = prices.to_struct("prices")
pstruct


prices
struct[8]
"{2020-02-03 00:00:00 EST,73.753387,75.980774,73.24926,74.810127,173788400,0.0,0.0}"
"{2020-02-04 00:00:00 EST,76.421898,77.471367,76.014717,77.279892,136616400,0.0,0.0}"
"{2020-02-05 00:00:00 EST,78.411736,78.71228,77.304108,77.910034,118826800,0.0,0.0}"
"{2020-02-06 00:00:00 EST,78.181495,78.823776,77.62162,78.82135,105425600,0.0,0.0}"
"{2020-02-07 00:00:00 EST,78.318439,78.568673,77.256767,77.749947,117684000,0.1925,0.0}"
…
"{2025-01-27 00:00:00 EST,224.020004,232.149994,223.979996,229.860001,94863400,0.0,0.0}"
"{2025-01-28 00:00:00 EST,230.850006,240.190002,230.809998,238.259995,75707600,0.0,0.0}"
"{2025-01-29 00:00:00 EST,234.119995,239.860001,234.009995,239.360001,45486100,0.0,0.0}"


In [6]:
def my_sma(series, period: int = 20):
    print(type(series).__name__)
    return calc_sma(series, period)

def my_atr(prices, period: int = 14):
    print(type(prices).__name__)
    if hasattr(prices, 'dtype') and prices.dtype.__class__.__name__ == 'Struct':
        print("struct ...")
        prices = prices.struct
    return calc_atr(prices, period)

def my_macd(series):
    print(type(series).__name__)
    result = calc_macd(series, wrap=True)
    if hasattr(result, 'to_struct'):
        return result.to_struct("macd")


result = prices.with_columns(
    col('close').map_batches(my_sma).alias("sma"),
    col('close').map_batches(my_macd).alias("macd"),
    pl.struct(pl.all()).map_batches(my_atr).alias("atr")
#    col('close').map_batches(my_macd)
#    sma = col('close').map_batches(my_sma)
#     atr = pl.struct(pl.all()).map_batches(my_atr)
#    macd = col('close').map_batches(my_macd)
#    atr = pl.all().map_batches(my_atr)
#    MACD() @ col("close").map_batches(calc_macd),
#    atr = ATR() @ pl.all()
#    sma = col('close').map_batches(SMA(20))
#    atr = ATR() @ pl.all()
#    sma = SMA(20) @ pl.all()
#    sma = SMA(20) @ col('close')
)

result

SeriesSeries
Series
struct ...



date,open,high,low,close,volume,dividends,stock splits,sma,macd,atr
"datetime[ns, America/New_York]",f64,f64,f64,f64,i64,f64,f64,f64,struct[3],f64
2020-02-03 00:00:00 EST,73.753387,75.980774,73.24926,74.810127,173788400,0.0,0.0,NaN,"{NaN,NaN,NaN}",NaN
2020-02-04 00:00:00 EST,76.421898,77.471367,76.014717,77.279892,136616400,0.0,0.0,NaN,"{NaN,NaN,NaN}",NaN
2020-02-05 00:00:00 EST,78.411736,78.71228,77.304108,77.910034,118826800,0.0,0.0,NaN,"{NaN,NaN,NaN}",NaN
2020-02-06 00:00:00 EST,78.181495,78.823776,77.62162,78.82135,105425600,0.0,0.0,NaN,"{NaN,NaN,NaN}",NaN
2020-02-07 00:00:00 EST,78.318439,78.568673,77.256767,77.749947,117684000,0.1925,0.0,NaN,"{NaN,NaN,NaN}",NaN
…,…,…,…,…,…,…,…,…,…,…
2025-01-27 00:00:00 EST,224.020004,232.149994,223.979996,229.860001,94863400,0.0,0.0,237.887999,"{-5.227918,-3.582608,-1.64531}",5.552272
2025-01-28 00:00:00 EST,230.850006,240.190002,230.809998,238.259995,75707600,0.0,0.0,236.849999,"{-4.234714,-3.713029,-0.521684}",5.893539
2025-01-29 00:00:00 EST,234.119995,239.860001,234.009995,239.360001,45486100,0.0,0.0,236.038499,"{-3.320554,-3.634534,0.313981}",5.890429


In [5]:
sma = calc_sma(prices['close'], 20)
sma

array([         nan,          nan,          nan, ..., 236.03849945,
       235.30799942, 234.58699951])

In [6]:
pl.Series(values=sma, name=None)

""
f64
NaN
NaN
NaN
NaN
NaN
…
237.887999
236.849999
236.038499


In [24]:
MACD() @ prices

macd,macdsignal,macdhist
f64,f64,f64
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
…,…,…
-5.227918,-3.582608,-1.64531
-4.234714,-3.713029,-0.521684
-3.320554,-3.634534,0.313981


In [25]:
ATR() @ prices

""
f64
NaN
NaN
NaN
NaN
NaN
…
5.552272
5.893539
5.890429


In [26]:
SMA(20) @ prices['close']

""
f64
NaN
NaN
NaN
NaN
NaN
…
237.887999
236.849999
236.038499
